In [1]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import getpass
import pandas as pd

In [3]:
client_id=str(getpass.getpass('enter your client id?')) #ee2037b6235f4d19a75129a108d45a28
client_secret=str(getpass.getpass('enter your clientsecret?')) #92bfc64c53914d54a30f44553174fe89

enter your client id?········
enter your clientsecret?········


In [4]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret))

# 1. function to get the tracks of playlists (with more than 100 songs)

In [5]:
def get_playlist_tracks_long(playlist_id): #get tracks for playlists > 100
    playlist=sp.playlist_tracks(playlist_id)
    tracks=playlist['items']
    while playlist['next']:
        playlist=sp.next(playlist)
        tracks.extend(playlist['items'])
    return tracks

# 2. function to get the audio features of the songs in a playlist in a dataframe 

In [13]:
def audiofeat_from_playlistLONG(playlist_id):
    tracks = get_playlist_tracks_long(playlist_id)
    
    audio_features_tot = []
    playlist_uris_chunked = [] 
    playlist_uris=[]
    
    for item in tracks: #append list with uri of each song in the playlist
        playlist_uris.append(item['track']['uri'])
    
    
    chunk_size = 100
    for i in range(0, len(playlist_uris), chunk_size):
        playlist_uris_chunked.append(playlist_uris[i:i+chunk_size])
    
    for uri in playlist_uris_chunked:
        audio_features_temp = sp.audio_features(tracks=uri)
        audio_features_tot.append(audio_features_temp) #extract audio features
    
     
    af_df_1 = pd.DataFrame(audio_features_tot[0])
    
    for item in audio_features_tot[1:]:
        af_df_temp_1 = pd.DataFrame(item)
        af_df_1 = pd.concat([af_df_1, af_df_temp_1])
    
    return (af_df_1) 

In [14]:
# List of playlist from where to get the songs from
playlists2 = ['37i9dQZF1DWXRqgorJj26U','3Ho3iO0iJykgEQNbjB2sic','37i9dQZF1DX8ucPPHyzDUE', '37i9dQZF1DX55yuR78Invt', 
              '37i9dQZF1DWWxPM4nWdhyI', '37i9dQZF1DZ06evO161ZV6', '37i9dQZF1DZ06evO2zJOi2', '37i9dQZF1DZ06evO2ZTWSp', 
              '37i9dQZF1DX7JVsTTEkyUT', '37i9dQZF1E4k8iFIrwwqkW', '37i9dQZF1E4BoMLQnbv68l','37i9dQZF1E4BHGOS34BXBo', 
              '37i9dQZF1E4mHJ8YzKWXiJ', '37i9dQZF1E4CWDuYF7GiUo']

In [20]:
audio_tot_df = audiofeat_from_playlistLONG(playlists2[0]) # getting the first dataframe

In [21]:
for item in playlists2[1:]: #getting the final dataframe
    temp_df = audiofeat_from_playlistLONG(item) 
    #af_df_temp3 = pd.DataFrame(audio_features_playlist)
    audio_tot_df = pd.concat([audio_tot_df, temp_df])

In [22]:
audio_tot_df.reset_index(level = None,inplace = True)

In [23]:
audio_tot_df

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,0.615,0.910,11,-4.248,1,0.0347,0.500000,0.511000,0.281,0.770,111.013,audio_features,4hq0S6wznq7SHDyMOFXL9i,spotify:track:4hq0S6wznq7SHDyMOFXL9i,https://api.spotify.com/v1/tracks/4hq0S6wznq7S...,https://api.spotify.com/v1/audio-analysis/4hq0...,226333,4
1,1,0.481,0.262,7,-15.730,1,0.0414,0.735000,0.011400,0.832,0.375,122.861,audio_features,6mFkJmJqdDVQ1REhVfGgd1,spotify:track:6mFkJmJqdDVQ1REhVfGgd1,https://api.spotify.com/v1/tracks/6mFkJmJqdDVQ...,https://api.spotify.com/v1/audio-analysis/6mFk...,334744,4
2,2,0.502,0.912,1,-4.556,1,0.0564,0.000025,0.000173,0.106,0.720,116.761,audio_features,5ghIJDpPoe3CfHMGu71E6T,spotify:track:5ghIJDpPoe3CfHMGu71E6T,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,https://api.spotify.com/v1/audio-analysis/5ghI...,301920,4
3,3,0.273,0.952,11,-8.762,1,0.0843,0.016900,0.011100,0.142,0.472,100.271,audio_features,3YBZIN3rekqsKxbJc9FZko,spotify:track:3YBZIN3rekqsKxbJc9FZko,https://api.spotify.com/v1/tracks/3YBZIN3rekqs...,https://api.spotify.com/v1/audio-analysis/3YBZ...,405640,4
4,4,0.933,0.528,5,-6.472,0,0.1620,0.112000,0.329000,0.163,0.756,109.975,audio_features,57JVGBtBLCfHw2muk5416J,spotify:track:57JVGBtBLCfHw2muk5416J,https://api.spotify.com/v1/tracks/57JVGBtBLCfH...,https://api.spotify.com/v1/audio-analysis/57JV...,214653,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,45,0.768,0.644,2,-8.378,1,0.3260,0.387000,0.000000,0.236,0.453,123.892,audio_features,0O8XuLi337OtgoXUgKORQ5,spotify:track:0O8XuLi337OtgoXUgKORQ5,https://api.spotify.com/v1/tracks/0O8XuLi337Ot...,https://api.spotify.com/v1/audio-analysis/0O8X...,415792,4
1399,46,0.772,0.803,1,-3.035,1,0.0468,0.188000,0.000000,0.434,0.853,94.981,audio_features,6U2NuJILEOEpW2tVCuqKHb,spotify:track:6U2NuJILEOEpW2tVCuqKHb,https://api.spotify.com/v1/tracks/6U2NuJILEOEp...,https://api.spotify.com/v1/audio-analysis/6U2N...,242160,4
1400,47,0.802,0.787,7,-6.181,0,0.0599,0.083200,0.000000,0.153,0.446,94.026,audio_features,53v2u51nq3CxiKUNphRzP1,spotify:track:53v2u51nq3CxiKUNphRzP1,https://api.spotify.com/v1/tracks/53v2u51nq3Cx...,https://api.spotify.com/v1/audio-analysis/53v2...,167975,4
1401,48,0.790,0.631,8,-5.549,0,0.0425,0.437000,0.000000,0.158,0.889,94.039,audio_features,1LiN0Z98FkR1t0m8KmLcAH,spotify:track:1LiN0Z98FkR1t0m8KmLcAH,https://api.spotify.com/v1/tracks/1LiN0Z98FkR1...,https://api.spotify.com/v1/audio-analysis/1LiN...,190005,4


In [37]:
#adding the name of the song
name = []

for item in audio_tot_df['uri']:
    name.append(sp.track(item)['name'])

In [39]:
audio_tot_df['name'] = name

In [40]:
audio_tot_df

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,name
0,0,0.615,0.910,11,-4.248,1,0.0347,0.500000,0.511000,0.281,0.770,111.013,audio_features,4hq0S6wznq7SHDyMOFXL9i,spotify:track:4hq0S6wznq7SHDyMOFXL9i,https://api.spotify.com/v1/tracks/4hq0S6wznq7S...,https://api.spotify.com/v1/audio-analysis/4hq0...,226333,4,Tumbling Dice
1,1,0.481,0.262,7,-15.730,1,0.0414,0.735000,0.011400,0.832,0.375,122.861,audio_features,6mFkJmJqdDVQ1REhVfGgd1,spotify:track:6mFkJmJqdDVQ1REhVfGgd1,https://api.spotify.com/v1/tracks/6mFkJmJqdDVQ...,https://api.spotify.com/v1/audio-analysis/6mFk...,334744,4,Wish You Were Here
2,2,0.502,0.912,1,-4.556,1,0.0564,0.000025,0.000173,0.106,0.720,116.761,audio_features,5ghIJDpPoe3CfHMGu71E6T,spotify:track:5ghIJDpPoe3CfHMGu71E6T,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,https://api.spotify.com/v1/audio-analysis/5ghI...,301920,4,Smells Like Teen Spirit
3,3,0.273,0.952,11,-8.762,1,0.0843,0.016900,0.011100,0.142,0.472,100.271,audio_features,3YBZIN3rekqsKxbJc9FZko,spotify:track:3YBZIN3rekqsKxbJc9FZko,https://api.spotify.com/v1/tracks/3YBZIN3rekqs...,https://api.spotify.com/v1/audio-analysis/3YBZ...,405640,4,Paradise City
4,4,0.933,0.528,5,-6.472,0,0.1620,0.112000,0.329000,0.163,0.756,109.975,audio_features,57JVGBtBLCfHw2muk5416J,spotify:track:57JVGBtBLCfHw2muk5416J,https://api.spotify.com/v1/tracks/57JVGBtBLCfH...,https://api.spotify.com/v1/audio-analysis/57JV...,214653,4,Another One Bites The Dust - Remastered 2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,45,0.768,0.644,2,-8.378,1,0.3260,0.387000,0.000000,0.236,0.453,123.892,audio_features,0O8XuLi337OtgoXUgKORQ5,spotify:track:0O8XuLi337OtgoXUgKORQ5,https://api.spotify.com/v1/tracks/0O8XuLi337Ot...,https://api.spotify.com/v1/audio-analysis/0O8X...,415792,4,"Ella Y Yo (feat. Farruko, Ozuna, Arcangel, Anu..."
1399,46,0.772,0.803,1,-3.035,1,0.0468,0.188000,0.000000,0.434,0.853,94.981,audio_features,6U2NuJILEOEpW2tVCuqKHb,spotify:track:6U2NuJILEOEpW2tVCuqKHb,https://api.spotify.com/v1/tracks/6U2NuJILEOEp...,https://api.spotify.com/v1/audio-analysis/6U2N...,242160,4,Dura - Remix
1400,47,0.802,0.787,7,-6.181,0,0.0599,0.083200,0.000000,0.153,0.446,94.026,audio_features,53v2u51nq3CxiKUNphRzP1,spotify:track:53v2u51nq3CxiKUNphRzP1,https://api.spotify.com/v1/tracks/53v2u51nq3Cx...,https://api.spotify.com/v1/audio-analysis/53v2...,167975,4,Que Malo
1401,48,0.790,0.631,8,-5.549,0,0.0425,0.437000,0.000000,0.158,0.889,94.039,audio_features,1LiN0Z98FkR1t0m8KmLcAH,spotify:track:1LiN0Z98FkR1t0m8KmLcAH,https://api.spotify.com/v1/tracks/1LiN0Z98FkR1...,https://api.spotify.com/v1/audio-analysis/1LiN...,190005,4,Amanece
